## 1. Load Libraries

In [1]:
import pandas as pd
import pubchempy as pcp
import numpy as np
import os
import glob
import re
from pybatchclassyfire import *
import csv 
import time
import json
from pandas import json_normalize
import wget
import string
import urllib.parse
import openpyxl
import statistics
import sys
from itertools import chain
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem import PandasTools

INFO:rdkit:Enabling RDKit 2021.09.4 jupyter extensions


In [2]:
from Workflow_Python_Functions import one_candidate_selection
#from Workflow_Python_Functions import CandidateSelection_SimilarityandIdentity
from Workflow_Python_Functions import chemMN_CandidateSelection

## 2. Pre-process SpectralDB Results

In [3]:
def isNaN(string):
    return string != string

# Define scoring for all DBs
def HMDB_Scoring(db, i):
    if db['HMDBintScore'][i] >= 0.50 and db['HMDBmzScore'][i] >= 0.40 and db['HQMatchingPeaks'][i]/db['hQueryTotalPeaks'][i] >= 0.50:
        return True
    else:
        return False


def GNPS_Scoring(db, i):
    if db['GNPSintScore'][i] >= 0.50 and db['GNPSmzScore'][i] >= 0.40 and db['GQMatchingPeaks'][i]/db['gQueryTotalPeaks'][i] >= 0.50:
        return True
    else:
        return False


def MB_Scoring(db, i):
    if db['MBintScore'][i] >= 0.50 and db['MBmzScore'][i] >= 0.40 and db['MQMatchingPeaks'][i]/db['mQueryTotalPeaks'][i] >= 0.50:
        return True
    else:
        return False


In [4]:
#currently only these subsets are removed from the names from GNPS
matches = ["M+","[M", "M-", "2M", "M*" "20.0", "50.0", "30.0", "40.0", "60.0", "70.0", "eV", "Massbank"
           , "Spectral", "Match", "to", "from", "NIST14", "MoNA", '[IIN-based:',  '[IIN-based', 'on:', 'CCMSLIB00003136269]']

#open another csv path holding empty list, which will be filled 
#with post processed csv results
GNPScsvfiles2 = []

In [5]:
fls_g = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/spectral_dereplication/GNPS/gnps_results_for_file_6M285R540ID286.csv"

In [6]:
gnps_df = pd.read_csv(fls_g)
gnps_df = gnps_df.drop_duplicates(subset=['GNPSspectrumID'])

In [7]:
gnps_df = gnps_df[0:30]

In [17]:
for i, row in gnps_df.iterrows():
    # if compound name is present

    if GNPS_Scoring(gnps_df, i):
        if not isNaN(gnps_df['GNPScompound_name'][i]):
            # split if there is a gap in the names

            string_chng = (gnps_df['GNPScompound_name'][i].split(" "))

            # create an empty list
            newstr = []

            # for each part of the string in the names
            chng = []
            
            for j in range(len(string_chng)):
                # check if the substrings are present in the matches and no - is present

                if not any(x in string_chng[j] for x in matches): #and not '-' == string_chng[j]:

                    # IF | and ! not in the substring
                    if '|' not in string_chng[j] or '!' not in string_chng[j]:

                        newstr.append(string_chng[j])
                    # if | present in the substring   
                    elif '|' in string_chng[j]:

                        #split the string
                        jlen = string_chng[j].split("|")
                        #how many substrings are left now
                        lst = len(jlen)-1
                        #append this to chng
                        chng.append(jlen[lst])
                        break

                        # now append chng to newstr            
            chng.append(' '.join(newstr))
            #save this as the correct name
            gnps_df.loc[i, "corr_names"] = chng[0]

            if not isNaN(gnps_df['GNPSSMILES'][i]):
                if chng == '':
                    break
                elif gnps_df['GNPSSMILES'][i].isalpha():
                    s = pcp.get_compounds(chng[0], 'name')
                    if s:
                        for comp in s:
                            gnps_df["GNPSSMILES"][i] = comp.isomeric_smiles
                    else:
                        gnps_df["GNPSSMILES"][i] = ''
        else:
            gnps_df["GNPSSMILES"][i] = ''
    else:
        gnps_df.drop([i], axis=0, inplace=True)
for k, row in gnps_df.iterrows():
                                                
    if isNaN(gnps_df['GNPSSMILES'][k]):

        if "[" in gnps_df['GNPScompound_name'][k].split(" ")[-1]:
            string_chng = (gnps_df['GNPScompound_name'][k].split("["))
            #print(gnps_df['GNPScompound_name'][i])

            keep_names = []
            for j in range(len(string_chng)-1):
                gnps_df.loc[k, "corr_names"] == string_chng[j]
                s = pcp.get_compounds(string_chng[j], 'name')

                if s:
                    for comp in s:
                        gnps_df["GNPSSMILES"][k] = comp.isomeric_smiles
                        gnps_df.loc[k, "GNPSformula"] = comp.molecular_formula
                        gnps_df.loc[k, "GNPSinchi"] = Chem.MolToInchi(Chem.MolFromSmiles(comp.isomeric_smiles))


                else:
                    gnps_df["GNPSSMILES"][k] = ''
                    gnps_df.loc[k, "GNPSformula"] = ''
                    gnps_df.loc[k, "GNPSinchi"] = ''
    if not isNaN(gnps_df['GNPSSMILES'][k]):
        try:

            sx = pcp.get_compounds(gnps_df['GNPSSMILES'][k], 'smiles')
            gnps_df.loc[k, "GNPSinchi"] = Chem.MolToInchi(Chem.MolFromSmiles(comp.isomeric_smiles))
            if sx:
                sx = str(sx)
                comp = pcp.Compound.from_cid([int(x) for x in re.findall(r'\b\d+\b', sx)])
                gnps_df.loc[k, 'GNPSformula'] = comp.molecular_formula

        except:
            gnps_df.loc[k, "GNPSformula"] = ''
            gnps_df.loc[k, "GNPSinchi"] = ''


DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/JSON
DEBUG:pubchempy:Request data: b'smiles=Cc%28cc1C%28c2cc%28OC%29cc%28O%29c22%29%3DO%29cc%28O%29c1C2%3DO'
DEBUG:pubchempy:Created Compound(10639)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/JSON
DEBUG:pubchempy:Request data: b'cid=10639'
DEBUG:pubchempy:Created Compound(10639)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/JSON
DEBUG:pubchempy:Request data: b'smiles=OC1%3DC%28%5BC%40%40H%5D2O%5BC%40H%5D%28CO%29%5BC%40%40H%5D%28O%29%5BC%40H%5D%28O%5BC%40%40H%5D3O%5BC%40H%5D%28CO%29%5BC%40%40H%5D%28O%29%5BC%40H%5D%28O%29%5BC%40H%5D3O%29%5BC%40H%5D2O%29C%28O%29%3DC4C%28OC%28C5%3DCC%3DC%28OC%29C%3DC5%29%3DCC4%3DO%29%3DC1'
DEBUG:pubchempy:Created Compound(163121066)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/JSON
DEBUG:pubchempy:Request data: b'cid=163121066'
DEBUG:pubchempy:Created 

DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/JSON
DEBUG:pubchempy:Request data: b'smiles=COc1ccc%28-c2coc3cc%28O%29cc%28O%29c3c2%3DO%29cc1'
DEBUG:pubchempy:Created Compound(5280373)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/JSON
DEBUG:pubchempy:Request data: b'cid=5280373'
DEBUG:pubchempy:Created Compound(5280373)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/JSON
DEBUG:pubchempy:Request data: b'smiles=COc1ccc%28-c2coc3cc%28O%29cc%28O%29c3c2%3DO%29cc1'
DEBUG:pubchempy:Created Compound(5280373)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/JSON
DEBUG:pubchempy:Request data: b'cid=5280373'
DEBUG:pubchempy:Created Compound(5280373)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/JSON
DEBUG:pubchempy:Request data: b'smiles=COc1ccc%28-c2coc3cc%28O%29cc%28O%29c3c2%3DO%29cc1'
DEBUG:pubchempy:Cr

In [22]:
gnps_df

Unnamed: 0  GNPSmax_similarity  GNPSmzScore  GNPSintScore  \
14          15            0.936338     0.666667           1.0   
15          16            0.936338     0.666667           1.0   
16          17            0.936338     0.666667           1.0   
17          18            0.936338     0.666667           1.0   
18          19            0.936338     0.666667           1.0   
19          20            0.936338     0.666667           1.0   
20          21            0.936338     0.666667           1.0   
21          22            0.936338     0.666667           1.0   
22          23            0.936338     0.666667           1.0   
23          24            0.936338     0.666667           1.0   
24          25            0.936338     0.666667           1.0   
25          26            0.936338     0.666667           1.0   
26          27            0.936338     0.666667           1.0   
27          28            0.936338     0.666667           1.0   
28          29            0.936338     0.666667           1.0   
29          30            0.936338     0.666667           1.0   
30          31            0.936338     0.666667           1.0   
31          32            0.936338     0.666667           1.0   
32          33            0.936338     0.666667           1.0   
34          35            0.936338     0.666667           1.0   
39          40            0.936338     0.666667           1.0   
40          41            0.936338     0.666667           1.0   
41          42            0.936338     0.666667           1.0   
42          43            0.936338     0.666667           1.0   
43          44            0.936338     0.666667           1.0   
44          45            0.936338     0.666667           1.0   
45          46            0.936338     0.666667           1.0   
46          47            0.936338     0.666667           1.0   
47          48            0.936338     0.666667           1.0   

    GQMatchingPeaks  GNPSTotalPeaks  gQueryTotalPeaks  \
14              1.0             1.0               2.0   
15              1.0             1.0               2.0   
16              1.0             1.0               2.0   
17              1.0             1.0               2.0   
18              1.0             1.0               2.0   
19              1.0             1.0               2.0   
20              1.0             1.0               2.0   
21              1.0             1.0               2.0   
22              1.0             1.0               2.0   
23              1.0             1.0               2.0   
24              1.0             1.0               2.0   
25              1.0             1.0               2.0   
26              1.0             1.0               2.0   
27              1.0             1.0               2.0   
28              1.0             1.0               2.0   
29              1.0             1.0               2.0   
30              1.0             1.0               2.0   
31              1.0             1.0               2.0   
32              1.0             1.0               2.0   
34              1.0             1.0               2.0   
39              1.0             1.0               2.0   
40              1.0             1.0               2.0   
41              1.0             1.0               2.0   
42              1.0             1.0               2.0   
43              1.0             1.0               2.0   
44              1.0             1.0               2.0   
45              1.0             1.0               2.0   
46              1.0             1.0               2.0   
47              1.0             1.0               2.0   

                                           GNPSSMILES      GNPSspectrumID  \
14            Cc(cc1C(c2cc(OC)cc(O)c22)=O)cc(O)c1C2=O  CCMSLIB00005724546   
15  OC1=C([C@@H]2O[C@H](CO)[C@@H](O)[C@H](O[C@@H]3...  CCMSLIB00005788115   
16  COC1=CC=C(C=C1)C2=COC3=CC(OC4OC(COC5OC(C)C(O)C...  CCMSLIB00000848979   
17  COC1=CC=C(C=C1)C2CC(=O)C3=C(O2

In [23]:
gnps_df.to_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/spectral_dereplication/GNPS/gnps_results_for_file_6M285R540ID286proc.csv")

In [8]:
gnps_df = pd.read_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/spectral_dereplication/GNPS/gnps_results_for_file_6M285R540ID286proc.csv")

In [25]:
input_dir = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R"

In [31]:
#download SDF structures
#os.system("wget -P " + input_dir + " https://hmdb.ca/system/downloads/current/structures.zip")
#os.system("unzip "+ input_dir + "/structures.zip" + " -d " + input_dir)
# Load the sdf
dframe = PandasTools.LoadSDF((input_dir+"/structures.sdf"),
                         idName='HMDB_ID',smilesName='SMILES',
                         molColName='Molecule', includeFingerprints=False)


HMDBcsvfiles2 = []

RDKit ERROR: [19:38:20] ERROR: Explicit valence for atom # 7 N, 4, is greater than permitted
[19:38:20] ERROR: Explicit valence for atom # 7 N, 4, is greater than permitted
[20:03:44] Explicit valence for atom # 12 N, 4, is greater than permitted
RDKit ERROR: [20:03:44] Explicit valence for atom # 12 N, 4, is greater than permitted
RDKit ERROR: [20:03:44] ERROR: Could not sanitize molecule ending on line 102562
[20:03:44] ERROR: Could not sanitize molecule ending on line 102562
RDKit ERROR: [20:03:44] ERROR: Explicit valence for atom # 12 N, 4, is greater than permitted
[20:03:44] ERROR: Explicit valence for atom # 12 N, 4, is greater than permitted
[20:03:46] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit ERROR: [20:03:46] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit ERROR: [20:03:46] ERROR: Could not sanitize molecule ending on line 208820
[20:03:46] ERROR: Could not sanitize molecule ending on line 208820
RDKit WARNING: [20:03:49] Warning:

RDKit WARNING: [20:14:53] Warning: conflicting stereochemistry at atom 10 ignored. by rule 1a.
[20:14:53] Warning: conflicting stereochemistry at atom 10 ignored. by rule 1a.
RDKit WARNING: [20:14:53] Warning: conflicting stereochemistry at atom 10 ignored. by rule 1a.
[20:14:53] Warning: conflicting stereochemistry at atom 10 ignored. by rule 1a.
RDKit ERROR: [20:12:01] ERROR: moving to the beginning of the next molecule
[20:12:01] ERROR: moving to the beginning of the next molecule
[20:14:54] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit ERROR: [20:14:54] Explicit valence for atom # 1 N, 4, is greater than permitted
RDKit ERROR: [20:14:54] ERROR: Could not sanitize molecule ending on line 53009431
[20:14:54] ERROR: Could not sanitize molecule ending on line 53009431
RDKit WARNING: [20:14:54] WARNING: not removing hydrogen atom without neighbors
[20:14:54] WARNING: not removing hydrogen atom without neighbors
RDKit ERROR: [20:14:54] ERROR: Explicit valence for at

In [32]:
fls_h = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/spectral_dereplication/HMDB/hmdb_results_for_file_6M285R540ID286.csv"

In [33]:
hmdb_df = pd.read_csv(fls_h)
hmdb_df = hmdb_df.drop_duplicates(subset=['HMDBcompoundID'])

In [34]:
# merge on basis of id, frame and hmdb result files
SmilesHM = pd.merge(hmdb_df, dframe, left_on=hmdb_df.HMDBcompoundID, right_on=dframe.DATABASE_ID)


for i, row in hmdb_df.iterrows():
    if HMDB_Scoring(hmdb_df, i):

        for j, row in SmilesHM.iterrows():

            # where index for both match, add the name and SMILES
            if hmdb_df['HMDBcompoundID'][i]== SmilesHM['HMDBcompoundID'][j]:
                hmdb_df.loc[i, 'HMDBSMILES'] = SmilesHM['SMILES'][j]#add SMILES
                hmdb_df.loc[i, 'HMDBcompound_name'] = SmilesHM["GENERIC_NAME"][j]#add name
                hmdb_df.loc[i, 'HMDBformula'] = SmilesHM["FORMULA"][j]#add formula
                hmdb_df.loc[i, 'HMDBinchi'] = Chem.MolToInchi(Chem.MolFromSmiles(SmilesHM['SMILES'][j]))
    else:
        hmdb_df.drop([i], axis=0, inplace=True)

In [83]:
hmdb_df = hmdb_df[0:30]

In [41]:
hmdb_df[0:30].to_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/spectral_dereplication/HMDB/hmdb_results_for_file_6M285R540ID286proc.csv")

In [9]:
hmdb_df = pd.read_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/spectral_dereplication/HMDB/hmdb_results_for_file_6M285R540ID286proc.csv")

In [37]:
fls_m = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/spectral_dereplication/MassBank/mbank_results_for_file_6M285R540ID286.csv"

In [38]:
mbank_df = pd.read_csv(fls_m)
mbank_df = mbank_df.drop_duplicates(subset=['MBspectrumID'])
if len(mbank_df)>0:


    for i, row in mbank_df.iterrows():
        if MB_Scoring(mbank_df, i):

            inchiK = str(mbank_df["MBinchiKEY"][i])

            #extract inchikeys
            y = pcp.get_compounds(inchiK, 'inchikey')#compound based on inchikey

            for compound in y:

                #add smiles
                smles = compound.isomeric_smiles   
                mbank_df.loc[i, 'MBSMILES'] = smles
                mbank_df.loc[i, 'MBinchi'] =Chem.MolToInchi(Chem.MolFromSmiles(smles))
        else:
            mbank_df.drop([i], axis=0, inplace=True)



DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=FKTPLNFTYJEAAB-UHFFFAOYSA-N'
DEBUG:pubchempy:Created Compound(159926)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=HNMWDXUKPJZOQD-XPBDJYBLSA-N'
DEBUG:pubchempy:Created Compound(3084308)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=DANYIYRPLHHOCZ-UHFFFAOYSA-N'
DEBUG:pubchempy:Created Compound(5280442)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=HMUJXQRRKBLVOO-AWEZNQCLSA-N'
DEBUG:pubchempy:Created Compound(160481)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=DANYIYRPLHHOCZ-UHFFFAOYSA-N'
DEBUG:pubchemp

DEBUG:pubchempy:Request data: b'inchikey=WUADCCWRTIWANL-UHFFFAOYSA-N'
DEBUG:pubchempy:Created Compound(5280373)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=NLAWPKPYBMEWIR-UHFFFAOYSA-N'
DEBUG:pubchempy:Created Compound(4463283)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=NLAWPKPYBMEWIR-UHFFFAOYSA-N'
DEBUG:pubchempy:Created Compound(4463283)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=HWRDCYOHJBCWGW-UHFFFAOYSA-N'
DEBUG:pubchempy:Created Compound(74259754)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=MLWDGPFGTFOLRJ-CUVHLRMHSA-N'
DEBUG:pubchempy:Created Compound(5317385)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/r

DEBUG:pubchempy:Request data: b'inchikey=RMCRQBAILCLJGU-UHFFFAOYSA-N'
DEBUG:pubchempy:Created Compound(85705)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=MLWDGPFGTFOLRJ-CUVHLRMHSA-N'
DEBUG:pubchempy:Created Compound(5317385)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=HMUJXQRRKBLVOO-UHFFFAOYSA-N'
DEBUG:pubchempy:Created Compound(5118250)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=RMCRQBAILCLJGU-UHFFFAOYSA-N'
DEBUG:pubchempy:Created Compound(85705)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/JSON
DEBUG:pubchempy:Request data: b'inchikey=YFVGIJBUXMQFOF-PJOVQGMDSA-N'
DEBUG:pubchempy:Created Compound(5317025)
DEBUG:pubchempy:Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/p

In [10]:
mbank_df[0:30].to_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/spectral_dereplication/MassBank/mbank_results_for_file_6M285R540ID286.csv")

NameError: name 'mbank_df' is not defined

In [11]:
mbank_df = pd.read_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/spectral_dereplication/MassBank/mbank_results_for_file_6M285R540ID286.csv")

In [82]:
mbank_df = mbank_df[0:30]

## 3. MCSS For Spectral DBs

In [7]:
# Describe the heavy atoms to be considered for MCSS
heavy_atoms = ['C', 'N', 'P', 'O', 'S']

In [8]:
gnpsproc = pd.read_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/spectral_dereplication/GNPS/gnps_results_for_file_6M285R540ID286.csv") 


if len(gnpsproc)>0:
    G_Smiles = gnpsproc["GNPSSMILES"]
    G_Smiles = list(filter(None, G_Smiles))
    #print(G_Smiles)
    #create empty list of GNPS top smiles
    GNPS_Mol = []
    # extract only the InChI of the top 5
    for j in list(G_Smiles):
        if not isNaN(j):
            #print(type(j))
            mol2 = Chem.MolFromSmiles(j)
            GNPS_Mol.append(mol2)

    if len(GNPS_Mol) >= 2:
        res = rdFMCS.FindMCS(GNPS_Mol)
        sm_res = Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString))
        # if there are atleast 3 heavy atoms in the MCSS, then add it to the result file
        elem = [ele for ele in heavy_atoms if(ele in sm_res)]
        if elem and len(sm_res)>=3:
            print(res.smartsString)
            print(Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString)))

[#6](:,-[#8]):,-[#6]:,-[#6]:,-[#6]:,-[#6]:,-[#6]
C:C:C:C:C:C:O


In [9]:
gnps_mcss = "C:C:C:C:C:C:O"

In [10]:
hmdbproc = pd.read_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/spectral_dereplication/HMDB/hmdb_results_for_file_6M285R540ID286proc.csv") 

if len(hmdbproc)>0:
    H_Smiles = hmdbproc["HMDBSMILES"]
    H_Smiles = list(filter(None, H_Smiles))

    HMDB_Mol = []
    # extract only the InChI of the top 5
    for j in list(H_Smiles):
        if not isNaN(j):

            mol2 = Chem.MolFromSmiles(j)
            HMDB_Mol.append(mol2)

    if len(HMDB_Mol) >= 2:
        res = rdFMCS.FindMCS(HMDB_Mol)
        sm_res = Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString))
        # if there are atleast 3 heavy atoms in the MCSS, then add it to the result file
        elem = [ele for ele in heavy_atoms if(ele in sm_res)]
        if elem and len(sm_res)>=3:
            print(res.smartsString)
            print(Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString)))

[#8]-,:[#6]:[#6]:,-[#6]:,-[#6](-,:[#6]:,-[#6]:,-[#6]:,-[#6]:,-[#6]:,-[#6]:,-[#6]):,-[#6]
C:C:C:C:C:C:CC(:C):C:C:CO


In [11]:
hmdb_mcss = "C:C:C:C:C:C:CC(:C):C:C:CO"

In [12]:
mbankproc = pd.read_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/spectral_dereplication/MassBank/mbank_results_for_file_6M285R540ID286.csv") 

if len(mbankproc)>0:
    M_Smiles = mbankproc["MBSMILES"]
    M_Smiles = list(filter(None, M_Smiles))

    MB_Mol = []
    # extract only the InChI of the top 5
    for j in list(M_Smiles):
        if not isNaN(j):

            mol2 = Chem.MolFromSmiles(j)
            MB_Mol.append(mol2)

    if len(MB_Mol) >= 2:
        res = rdFMCS.FindMCS(MB_Mol)
        sm_res = Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString))
        # if there are atleast 3 heavy atoms in the MCSS, then add it to the result file
        elem = [ele for ele in heavy_atoms if(ele in sm_res)]
        if elem and len(sm_res)>=3:
            print(res.smartsString)
            print(Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString)))

[#6]-,:[#6]:,-[#6]:,-[#6]:[#6]:,-[#6]-,:[#6]-,:[#6]:,-[#6]:,-[#6](:,-[#6]:,-[#6](:,-[#6])-[#8])-[#8]
CC:C:C:C:CCC:C:C(O):C:C(:C)O


In [13]:
mbank_mcss = "CC:C:C:C:CCC:C:C(O):C:C(:C)O"

## 4. SIRIUS postprocessing

In [14]:
def str_can_score(db, i):
    if db['explainedIntensity'][i] >= exp_int and db['CSI:FingerIDScore'][i] >= csi_score:
        return True
    else:
        return False

In [15]:
exp_int = 0.70
csi_score = -150

In [16]:
sub_sub_dirALL_structure_can = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/insilico/SIRIUS/286_iso_MS1p_285.07858276_SIRIUS_param.json/0_286_iso_MS1p_285.07858276_SIRIUS_param_file_6M285R540ID286/structure_candidates.tsv"                   
sub_sub_dirALL_formula_can = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/insilico/SIRIUS/286_iso_MS1p_285.07858276_SIRIUS_param.json/0_286_iso_MS1p_285.07858276_SIRIUS_param_file_6M285R540ID286/formula_candidates.tsv" 
ALL_Canopus_csv = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/insilico/SIRIUS/286_iso_MS1p_285.07858276_SIRIUS_param.json/canopus_summary.tsv"

In [17]:
ALL_structure_csv = pd.read_csv(sub_sub_dirALL_structure_can, sep = "\t")
ALL_formula_csv = pd.read_csv(sub_sub_dirALL_formula_can, sep = "\t")
ALL_Canopus = pd.read_csv(ALL_Canopus_csv, sep = "\t")

In [75]:
# Add the structure and formula files together
for structure, rows in ALL_structure_csv.iterrows():
    for formula, rows in ALL_formula_csv.iterrows():
        if ALL_structure_csv["formulaRank"][structure] == ALL_formula_csv["rank"][formula]:
            ALL_structure_csv.loc[structure, 'SiriusScore'] = ALL_formula_csv['SiriusScore'][formula]
            ALL_structure_csv.loc[structure, 'numExplainedPeaks'] = ALL_formula_csv['numExplainedPeaks'][formula]
            ALL_structure_csv.loc[structure, 'explainedIntensity'] = ALL_formula_csv['explainedIntensity'][formula]
            ALL_structure_csv.loc[structure, "SuspectListEntry"] = "FALSE"
            if len(ALL_Canopus)>0:
                if ALL_formula_csv["molecularFormula"][formula] == ALL_Canopus["molecularFormula"][0]:
                    ALL_structure_csv.loc[structure, 'superclass'] = ALL_Canopus['superclass'][0]
                    ALL_structure_csv.loc[structure, 'class'] = ALL_Canopus['class'][0]
                    ALL_structure_csv.loc[structure, 'subclass'] = ALL_Canopus['subclass'][0]

for str_siriusA, row in ALL_structure_csv.iterrows():
    if not str_can_score(ALL_structure_csv, str_siriusA):
        ALL_structure_csv = ALL_structure_csv.drop(str_siriusA, inplace=False)

In [76]:
ALL_structure_csv

rank  formulaRank  CSI:FingerIDScore molecularFormula    adduct  \
0        1            1         -28.926681         C16H12O5  [M + H]+   
1        2            1         -33.715265         C16H12O5  [M + H]+   
2        3            1         -33.853965         C16H12O5  [M + H]+   
3        4            1         -42.884305         C16H12O5  [M + H]+   
4        5            1         -53.895936         C16H12O5  [M + H]+   
...    ...          ...                ...              ...       ...   
1149     9            2        -144.626032       C12H14N4O2  [M + K]+   
1150    10            2        -147.333654       C12H14N4O2  [M + K]+   
1151    11            2        -147.767235       C12H14N4O2  [M + K]+   
1152    12            2        -148.366742       C12H14N4O2  [M + K]+   
1153    13            2        -149.881513       C12H14N4O2  [M + K]+   

          InChIkey2D                                              InChI  \
0     WUADCCWRTIWANL  InChI=1S/C16H12O5/c1-20-11-4-2-9(3-5-11)12-8-2...   
1     YSINCDVRUMTOPK  InChI=1S/C16H12O5/c1-20-13-6-11(18)7-14-15(13)...   
2     KQMVAGISDHMXJJ  InChI=1S/C16H12O5/c1-20-11-6-13(18)15-14(7-11)...   
3     DXYUAIFZCFRPTH  InChI=1S/C16H12O5/c1-20-15-6-11-14(7-13(15)18)...   
4     GCWOYVFHJDNKIN  InChI=1S/C16H12O5/c1-20-10-4-2-9(3-5-10)12-8-2...   
...              ...                                                ...   
1149  UOLGETQPFJFJDU  InChI=1S/C12H14N4O2/c1-17-9-5-8(6-10(7-9)18-2)...   
1150  INVKQIMARQDBNH  InChI=1S/C12H14N4O2/c1-17-9-5-8(6-10(7-9)18-2)...   
1151  QDSQHTBNSDBQRA  InChI=1S/C12H14N4O2/c1-8-6-4-5-7-9(8)13-10-14-...   
1152  RMLFIHMTVRNTMS  InChI=1S/C12H14N4O2/c1-18-10-5-7(2-3-9(10)17)4...   
1153  CBZSMIPYVFQEPU  InChI=1S/C12H14N4O2/c1-17-8-4-3-5-9(18-2)11(8)...   

                                                   name  \
0                                             Biochanin   
1                                           Isoprunetin   
2                                              Prunetin   
3                                             Glycetein   
4                                               Texasin   
...                                                 ...   
1149                                                NaN   
1150                                                NaN   
1151                                                NaN   
1152  4-[(2,4-diaminopyrimidin-5-yl)methyl]-2-methox...   
1153                                                NaN   

                                             smiles     xlogp  \
0         COC1=CC=C(C=C1)C2=COC3=CC(=CC(=C3C2=O)O)O  3.000000   
1       COC1=CC(=CC2=C1C(=O)C(=CO2)C3=CC=C(C=C3)O)O  2.400000   
2       COC1=CC(=C2C(=C1)OC=C(C2=O)C3=CC=C(C=C3)O)O  3.000000   
3     COC1=C(C=C2C(=C1)C(=O)C(=CO2)C3=CC=C(C=C3)O)O  2.400000   
4         COC1=CC=C(C=C1)C2=COC3=CC(=C(C=C3C2=O)O)O  2.400000   
...                                             ...       ...   
1149             COC1=CC(=CC(=C1)NC2=NC=CC(=N2)N)OC  1.700000   
1150             COC1=CC(=CC(=C1)NC2=NC(=NC=C2)N)OC  1.700000   
1151               CC1=CC=CC=C1NC2=NC(=NC(=N2)OC)OC  2.800000   
1152             COC1=C(C=CC(=C1)CC2=CN=C(N=C2N)N)O  1.293727   
1153           COC1=C(C(=CC=C1)OC)C2=CC(=NC(=N2)N)N  1.100000   

                                             pubchemids  \
0     5280373;12255048;11778794;21673729;25203224;46...   
1                                               5748551   
2                                               5281804   
3                            5317750;11994876;101368112   
4                                               5281812   
...                                                 ...   
1149                                          113565042   
1150                                           80760100   
1151                                           12698548   
1152                                            3078765   
1153                                           57830262   

                             

In [77]:
ALL_structure_csv.to_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/insilico/SIRIUS/286_iso_MS1p_285.07858276_SIRIUS_param_results.csv")

In [12]:
sirius_df = pd.read_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/insilico/SIRIUS/286_iso_MS1p_285.07858276_SIRIUS_param_results.csv")

## 5. MCSS for SIRIUS

In [13]:
S_Smiles = ALL_structure_csv["smiles"]
#create empty list of MB top smiles
SIRIUS_Mol = []

# extract only the InChI of the top 5
for j in list(S_Smiles):
    mol2 = Chem.MolFromSmiles(j)
    SIRIUS_Mol.append(mol2)
if len(SIRIUS_Mol) >= 2:
    res = rdFMCS.FindMCS(SIRIUS_Mol)
    sm_res = Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString))
    # if there are atleast 3 heavy atoms in the MCSS, then add it to the result file
    elem = [ele for ele in heavy_atoms if(ele in sm_res)]
    if elem and len(sm_res)>=3:
        print(res.smartsString)
        print(Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString)))

NameError: name 'ALL_structure_csv' is not defined

In [20]:
sirius_mcss = "C1:C:C:C:C:C:1"

## 6. Candidate Selection

In [14]:
#sirius_df = pd.read_csv(sirius_csv)
sirius_df = sirius_df.drop_duplicates('smiles')

#mbank_df = pd.read_csv(mbank_csv)
mbank_df = mbank_df.drop_duplicates('MBSMILES')

#gnps_df = pd.read_csv(gnps_csv)
gnps_df = gnps_df.drop_duplicates('GNPSSMILES')

#hmdb_df = pd.read_csv(hmdb_csv)
hmdb_df = hmdb_df.drop_duplicates('HMDBSMILES')


In [15]:
mbank_df["rank_ids"] = ["M_" + str(s+1) for s in range(len(mbank_df))]

gnps_df["rank_ids"] = ["G_" + str(s+1) for s in range(len(gnps_df))]

hmdb_df["rank_ids"] = ["H_" + str(s+1) for s in range(len(hmdb_df))]

sirius_df["rank_ids"] = ["S_" + str(s) for s in sirius_df["rank"]]
sirius_df["Source"] = "SIRIUS"


source_l1 = [*(list(sirius_df["Source"])), 
   *(list(gnps_df["Source"]))
   ,*(list(mbank_df["Source"])),
            *(list(hmdb_df["Source"]))]

rank_l2 = [*(list(sirius_df["rank_ids"])), 
   *(list(gnps_df["rank_ids"]))
   ,*(list(mbank_df["rank_ids"])), 
          *(list(hmdb_df["rank_ids"]))]

smiles_l3 = [*(list(sirius_df["smiles"])), 
   *(list(gnps_df["GNPSSMILES"]))
   ,*(list(mbank_df["MBSMILES"])),
            *(list(hmdb_df["HMDBSMILES"]))]

sm = pd.DataFrame(list(zip(source_l1, rank_l2, smiles_l3)), columns = ["Source", "ranks", "SMILES"])   
df_edge = chemMN_CandidateSelection(sm)

In [35]:
df_edge.to_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/file_6M285R540ID286_ChemMNedges.tsv", sep='\t')

In [16]:
df = sm

In [17]:
sm

Source ranks                                         SMILES
0    SIRIUS   S_1      COC1=CC=C(C=C1)C2=COC3=CC(=CC(=C3C2=O)O)O
1    SIRIUS   S_2    COC1=CC(=CC2=C1C(=O)C(=CO2)C3=CC=C(C=C3)O)O
2    SIRIUS   S_3    COC1=CC(=C2C(=C1)OC=C(C2=O)C3=CC=C(C=C3)O)O
3    SIRIUS   S_4  COC1=C(C=C2C(=C1)C(=O)C(=CO2)C3=CC=C(C=C3)O)O
4    SIRIUS   S_5      COC1=CC=C(C=C1)C2=COC3=CC(=C(C=C3C2=O)O)O
..      ...   ...                                            ...
300    HMDB  H_24             COc1cc2c(cc1O)OCC(c1ccc(O)cc1)C2=O
301    HMDB  H_25            COc1cc2oc(=O)cc(-c3cccc(O)c3)c2cc1O
302    HMDB  H_26                                            NaN
303    HMDB  H_27             COc1c(O)cc(O)c2c1OC(c1ccccc1)CC2=O
304    HMDB  H_28            COc1ccc(-c2coc3cc(O)ccc3c2=O)c(O)c1

[305 rows x 3 columns]

In [18]:
df["SIRIUS"] = np.nan
df["GNPS"] = np.nan
df["MassBank"] = np.nan
df["HMDB"] = np.nan

In [19]:
tn_ident = 0.99

In [20]:
# for each SMILES in df
for smiles, rows in df.iterrows():

# If the source contains SIRIUS
#if Source == "SGHM" or Source == "SGM" or Source == "SGH" or Source == "SHM" or Source == "SG" or Source == "SM" or Source == "SH":
# sirius_df comes from within the function CandidateSelection_SimilarityandIdentity
    for sirius_i, row in sirius_df.iterrows():
        # calculate tanimoto
        try:
            ms = [Chem.MolFromSmiles(df["SMILES"][smiles]), Chem.MolFromSmiles(sirius_df["smiles"][sirius_i])]
            fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
            tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])
            # since we are dealing with idenity here so tanimoto of 0.99 is appropriate
            if tn >= tn_ident:

                # if SIRIUS is blank, add the SIRIUS id
                if isNaN(df["SIRIUS"][smiles]):

                    df.loc[smiles, "SIRIUS"] = sirius_df["rank_ids"][sirius_i]
                # if not empty, add SIRIUS id, with a comma
                else:
                    df.loc[smiles, "SIRIUS"] = str(df["SIRIUS"][smiles]) + ", "+ sirius_df["rank_ids"][sirius_i]


        except Exception as e:
            #print(e.string)
            pass


    # If the Source contains GNPS
    #if Source == "SGHM" or Source == "SGM" or Source == "SGH" or Source == "GHM" or Source == "SG" or Source == "GM" or Source == "GH":

    # gnps_df comes from within the function CandidateSelection_SimilarityandIdentity
    for gnps_i, row in gnps_df.iterrows():
        try:
            # calculate tanimoto
            ms = [Chem.MolFromSmiles(df["SMILES"][smiles]), Chem.MolFromSmiles(gnps_df["GNPSSMILES"][gnps_i])]
            fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
            tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])

            # since we are dealing with idenity here so tanimoto of 0.99 is appropriate
            if tn >= tn_ident:

                # if GNPS is blank, add the GNPS id
                if isNaN(df["GNPS"][smiles]):

                    df.loc[smiles, "GNPS"] = gnps_df["rank_ids"][gnps_i]
                # if not empty, add GNPS id, with a comma
                else:
                    df.loc[smiles, "GNPS"] = str(df["GNPS"][smiles]) + ", "+ gnps_df["rank_ids"][gnps_i]          

        except Exception as e:
            #print(e.string)
            pass

    # If the source contains MassBank
    #if Source == "SGHM" or Source == "SGM" or Source == "SHM" or Source == "GHM" or Source == "SM" or Source == "GM" or Source == "HM":
    # mbank_df comes from within the function CandidateSelection_SimilarityandIdentity
    for mbank_i, row in mbank_df.iterrows():
        try:
            # calculate tanimoto
            ms = [Chem.MolFromSmiles(df["SMILES"][smiles]), Chem.MolFromSmiles(mbank_df["MBSMILES"][mbank_i])]
            fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
            tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])

            # since we are dealing with idenity here so tanimoto of 0.99 is appropriate
            if tn >= tn_ident:

                # if MassBank is blank, add the MassBank id
                if isNaN(df["MassBank"][smiles]):

                    df.loc[smiles, "MassBank"] = mbank_df["rank_ids"][mbank_i]
                # if not empty, add MassBank id, with a comma
                else:
                    df.loc[smiles, "MassBank"] = str(df["MassBank"][smiles]) + ", "+ mbank_df["rank_ids"][mbank_i]


        except Exception as e:
            #print(e.string)
            pass


    # If the source contains HMDB
    #if Source == "SGHM" or Source == "SGH" or Source == "SHM" or Source == "GHM" or Source == "SH" or Source == "GH" or Source == "HM":


    for hmdb_i, row in hmdb_df.iterrows():
        try:
            # calculate tanimoto
            ms = [Chem.MolFromSmiles(df["SMILES"][smiles]), Chem.MolFromSmiles(hmdb_df["HMDBSMILES"][hmdb_i])]
            fps = [AllChem.GetMorganFingerprintAsBitVect(x,2, nBits=2048) for x in ms]
            tn = DataStructs.FingerprintSimilarity(fps[0],fps[1])

            # since we are dealing with idenity here so tanimoto of 0.99 is appropriate
            if tn >= tn_ident:

                # if HMDB is blank, add the HMDB id
                if isNaN(df["HMDB"][smiles]):

                    df.loc[smiles, "HMDB"] = hmdb_df["rank_ids"][hmdb_i]
                # if not empty, add HMDB id, with a comma
                else:
                    df.loc[smiles, "HMDB"] = str(df["HMDB"][smiles]) + ", "+ hmdb_df["rank_ids"][hmdb_i]


        except Exception as e:
            #print(e.string)
            pass

In [21]:
df

Source ranks                                         SMILES SIRIUS  GNPS  \
0    SIRIUS   S_1      COC1=CC=C(C=C1)C2=COC3=CC(=CC(=C3C2=O)O)O    S_1  G_13   
1    SIRIUS   S_2    COC1=CC(=CC2=C1C(=O)C(=CO2)C3=CC=C(C=C3)O)O    S_2   NaN   
2    SIRIUS   S_3    COC1=CC(=C2C(=C1)OC=C(C2=O)C3=CC=C(C=C3)O)O    S_3   NaN   
3    SIRIUS   S_4  COC1=C(C=C2C(=C1)C(=O)C(=CO2)C3=CC=C(C=C3)O)O    S_4   NaN   
4    SIRIUS   S_5      COC1=CC=C(C=C1)C2=COC3=CC(=C(C=C3C2=O)O)O    S_5   NaN   
..      ...   ...                                            ...    ...   ...   
300    HMDB  H_24             COc1cc2c(cc1O)OCC(c1ccc(O)cc1)C2=O    NaN   NaN   
301    HMDB  H_25            COc1cc2oc(=O)cc(-c3cccc(O)c3)c2cc1O  S_159   NaN   
302    HMDB  H_26                                            NaN    NaN   NaN   
303    HMDB  H_27             COc1c(O)cc(O)c2c1OC(c1ccccc1)CC2=O    NaN   NaN   
304    HMDB  H_28            COc1ccc(-c2coc3cc(O)ccc3c2=O)c(O)c1   S_21   NaN   

    MassBank  HMDB  
0       M_10   H_1  
1        NaN   NaN  
2        NaN   NaN  
3        NaN   NaN  
4        NaN   NaN  
..       ...   ...  
300      NaN  H_24  
301      NaN  H_25  
302      NaN   NaN  
303      NaN  H_27  
304      NaN  H_28  

[305 rows x 7 columns]

In [23]:
df.to_csv("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/Ponder_Bryo_results_RUN1_R/pos_05_summer_Polstr_5_1-A-5_01_17174/all_candidates.csv")

In [22]:
def add_count_column(df_one_candidate):
    # create new df only with the Sources column
    df = pd.DataFrame({"SIRIUS": df_one_candidate["SIRIUS"], 
                   "GNPS": df_one_candidate["GNPS"],
                   "MassBank": df_one_candidate["MassBank"], 
                   "HMDB": df_one_candidate["HMDB"]})
    
    # now check which rows have a value
    df = df.dropna(axis=0, how='all')
    df_one_candidate = df_one_candidate.dropna(subset=["SIRIUS", "GNPS", "HMDB", "MassBank"], how='all', inplace=True)
    index_SIRIUS = [x for x, row in df.iterrows() if not isNaN(df["SIRIUS"][x])]
    index_GNPS = [x for x, row in df.iterrows() if not isNaN(df["GNPS"][x])]
    index_MassBank = [x for x, row in df.iterrows() if not isNaN(df["MassBank"][x])]
    index_HMDB = [x for x, row in df.iterrows() if not isNaN(df["HMDB"][x])]
    
    # make a list of the rows
    list_of_indices = index_SIRIUS + index_GNPS + index_MassBank + index_HMDB

    # count how mnay times one of the rows is appearing and add count
    count_list = [[x, list_of_indices.count(x)] for x in set(list_of_indices)]
    # add this info to one_can
    df_one_candidate["Count"] = [count_list[x][1] for x in range(len(count_list))]
    # sort the list by count in descending order
    sorted_count_one_candidate = df_one_candidate.sort_values(by = "Count", ascending = False)
    return(sorted_count_one_candidate)

In [25]:
df_one1 = add_count_column(df)

TypeError: 'NoneType' object does not support item assignment

In [26]:
df_one = pd.DataFrame({"SIRIUS": df["SIRIUS"], 
                   "GNPS": df["GNPS"],
                   "MassBank": df["MassBank"], 
                   "HMDB": df["HMDB"]})
df_one

SIRIUS  GNPS MassBank  HMDB
0      S_1  G_13     M_10   H_1
1      S_2   NaN      NaN   NaN
2      S_3   NaN      NaN   NaN
3      S_4   NaN      NaN   NaN
4      S_5   NaN      NaN   NaN
..     ...   ...      ...   ...
299    NaN   NaN      NaN  H_23
300    NaN   NaN      NaN  H_24
301  S_159   NaN      NaN  H_25
303    NaN   NaN      NaN  H_27
304   S_21   NaN      NaN  H_28

[302 rows x 4 columns]

In [84]:
#df_one = df_one.dropna(axis=0, how='all')

In [85]:
#df_one

In [27]:
df.dropna(subset=["SIRIUS", "GNPS", "HMDB", "MassBank"], how='all', inplace=True)

In [28]:
index_SIRIUS = [x for x, row in df_one.iterrows() if not isNaN(df_one["SIRIUS"][x])]
index_GNPS = [x for x, row in df_one.iterrows() if not isNaN(df_one["GNPS"][x])]
index_MassBank = [x for x, row in df_one.iterrows() if not isNaN(df_one["MassBank"][x])]
index_HMDB = [x for x, row in df_one.iterrows() if not isNaN(df_one["HMDB"][x])]

In [29]:
# make a list of the rows
list_of_indices = index_SIRIUS + index_GNPS + index_MassBank + index_HMDB

In [30]:
count_list = [[x, list_of_indices.count(x)] for x in set(list_of_indices)]

In [31]:
#df.dropna(subset=["SIRIUS", "GNPS", "HMDB", "MassBank"], how='all', inplace=True)

In [32]:
df["Count"] = [count_list[x][1] for x in range(len(count_list))]

In [33]:
df

Source ranks                                         SMILES SIRIUS  GNPS  \
0    SIRIUS   S_1      COC1=CC=C(C=C1)C2=COC3=CC(=CC(=C3C2=O)O)O    S_1  G_13   
1    SIRIUS   S_2    COC1=CC(=CC2=C1C(=O)C(=CO2)C3=CC=C(C=C3)O)O    S_2   NaN   
2    SIRIUS   S_3    COC1=CC(=C2C(=C1)OC=C(C2=O)C3=CC=C(C=C3)O)O    S_3   NaN   
3    SIRIUS   S_4  COC1=C(C=C2C(=C1)C(=O)C(=CO2)C3=CC=C(C=C3)O)O    S_4   NaN   
4    SIRIUS   S_5      COC1=CC=C(C=C1)C2=COC3=CC(=C(C=C3C2=O)O)O    S_5   NaN   
..      ...   ...                                            ...    ...   ...   
299    HMDB  H_23           COc1c(O)cc2c(c1O)C(=O)CC(c1ccccc1)O2    NaN   NaN   
300    HMDB  H_24             COc1cc2c(cc1O)OCC(c1ccc(O)cc1)C2=O    NaN   NaN   
301    HMDB  H_25            COc1cc2oc(=O)cc(-c3cccc(O)c3)c2cc1O  S_159   NaN   
303    HMDB  H_27             COc1c(O)cc(O)c2c1OC(c1ccccc1)CC2=O    NaN   NaN   
304    HMDB  H_28            COc1ccc(-c2coc3cc(O)ccc3c2=O)c(O)c1   S_21   NaN   

    MassBank  HMDB  Count  
0       M_10   H_1      4  
1        NaN   NaN      1  
2        NaN   NaN      1  
3        NaN   NaN      1  
4        NaN   NaN      1  
..       ...   ...    ...  
299      NaN  H_23      1  
300      NaN  H_24      1  
301      NaN  H_25      2  
303      NaN  H_27      1  
304      NaN  H_28      2  

[302 rows x 8 columns]

In [34]:
sorted_count_one_candidate = df.sort_values(by = "Count", ascending = False)
sorted_count_one_candidate

Source  ranks                                           SMILES SIRIUS  \
0      SIRIUS    S_1        COC1=CC=C(C=C1)C2=COC3=CC(=CC(=C3C2=O)O)O    S_1   
277      HMDB    H_1              COc1ccc(-c2coc3cc(O)cc(O)c3c2=O)cc1    S_1   
276  MassBank   M_10        COC1=CC=C(C=C1)C2=COC3=CC(=CC(=C3C2=O)O)O    S_1   
264      GNPS   G_13              COc1ccc(-c2coc3cc(O)cc(O)c3c2=O)cc1    S_1   
109    SIRIUS  S_110    CC1=CC2=C(C(=C1)O)C(=O)C3=C(C2=O)C=C(C=C3O)OC  S_110   
..        ...    ...                                              ...    ...   
124    SIRIUS  S_125      CC1=CC(=C(C=C1C2=COC3=C(C2=O)C=CC(=C3)O)O)O  S_125   
125    SIRIUS  S_126  CC1=C(C(=C2C(=C1)C(=O)C3=CC(=CC(=C3C2=O)O)O)O)C  S_126   
126    SIRIUS  S_127    CC1=C(C(=O)OC2=C1C=CC(=C2)O)C3=C(C=C(C=C3)O)O  S_127   
127    SIRIUS  S_128    CC1=CC(=CC2=C1C(=O)C(=CO2)C3=CC(=C(C=C3)O)O)O  S_128   
151    SIRIUS  S_152    COC1=C(C(=C2C(=C1)C(=O)C(=CO2)C3=CC=CC=C3)O)O  S_152   

         GNPS MassBank HMDB  Count  
0        G_13     M_10  H_1      4  
277      G_13     M_10  H_1      4  
276      G_13     M_10  H_1      4  
264      G_13     M_10  H_1      4  
109  G_1, G_5      M_7  NaN      3  
..        ...      ...  ...    ...  
124       NaN      NaN  NaN      1  
125       NaN      NaN  NaN      1  
126       NaN      NaN  NaN      1  
127       NaN      NaN  NaN      1  
151       NaN      NaN  NaN      1  

[302 rows x 8 columns]

In [35]:
df_count_4 = df[df["Count"]==4]

In [36]:
df_count_4

Source ranks                                     SMILES SIRIUS  GNPS  \
0      SIRIUS   S_1  COC1=CC=C(C=C1)C2=COC3=CC(=CC(=C3C2=O)O)O    S_1  G_13   
264      GNPS  G_13        COc1ccc(-c2coc3cc(O)cc(O)c3c2=O)cc1    S_1  G_13   
276  MassBank  M_10  COC1=CC=C(C=C1)C2=COC3=CC(=CC(=C3C2=O)O)O    S_1  G_13   
277      HMDB   H_1        COc1ccc(-c2coc3cc(O)cc(O)c3c2=O)cc1    S_1  G_13   

    MassBank HMDB  Count  
0       M_10  H_1      4  
264     M_10  H_1      4  
276     M_10  H_1      4  
277     M_10  H_1      4

In [46]:
[counts.split('_')[1] for counts in df_count_4["ranks"]]

['1', '13', '10', '1']

In [47]:
df_count_4["rank_num"] = [counts.split('_')[1] for counts in df_count_4["ranks"]]

In [50]:
df_count_4 = df_count_4.sort_values(by = "rank_num")

In [51]:
df_count_4[['Source','ranks', 'SMILES', 'Count']]

Source ranks                                     SMILES  Count
0      SIRIUS   S_1  COC1=CC=C(C=C1)C2=COC3=CC(=CC(=C3C2=O)O)O      4
277      HMDB   H_1        COc1ccc(-c2coc3cc(O)cc(O)c3c2=O)cc1      4
276  MassBank  M_10  COC1=CC=C(C=C1)C2=COC3=CC(=CC(=C3C2=O)O)O      4
264      GNPS  G_13        COc1ccc(-c2coc3cc(O)cc(O)c3c2=O)cc1      4

In [52]:
csv_download = df_count_4[['Source','ranks', 'SMILES', 'Count']]
csv_download

Source ranks                                     SMILES  Count
0      SIRIUS   S_1  COC1=CC=C(C=C1)C2=COC3=CC(=CC(=C3C2=O)O)O      4
277      HMDB   H_1        COc1ccc(-c2coc3cc(O)cc(O)c3c2=O)cc1      4
276  MassBank  M_10  COC1=CC=C(C=C1)C2=COC3=CC(=CC(=C3C2=O)O)O      4
264      GNPS  G_13        COc1ccc(-c2coc3cc(O)cc(O)c3c2=O)cc1      4

In [53]:
heavy_atoms = ['C', 'N', 'P', 'O', 'S']

In [56]:
Mol = []
# extract only the InChI of the top 5
for j in list(csv_download["SMILES"]):
    if not isNaN(j):
        print(type(j))
        mol2 = Chem.MolFromSmiles(j)
        Mol.append(mol2)

if len(Mol) >= 2:
    res = rdFMCS.FindMCS(Mol)
    sm_res = Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString))
    # if there are atleast 3 heavy atoms in the MCSS, then add it to the result file
    elem = [ele for ele in heavy_atoms if(ele in sm_res)]
    if elem and len(sm_res)>=3:
        print(Chem.MolToSmiles(Chem.MolFromSmarts(res.smartsString)))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
COC1:C:C:C(C2:C:O:C3:C:C(O):C:C(O):C:3:C:2=O):C:C:1


In [57]:
res.smartsString

'[#6]-[#8]-[#6]1:[#6]:[#6]:[#6](:[#6]:[#6]:1)-[#6]1:[#6]:[#8]:[#6]2:[#6](:[#6]:1=[#8]):[#6](:[#6]:[#6](:[#6]:2)-[#8])-[#8]'